In [214]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [234]:
# IMPORTING THE DATASET
d = pd.read_csv('m500_lap_time_rank_ver5.csv')
df = pd.DataFrame(data=d)

# FOR ROWS 4267 TO THE END, IT SEEMS LIKE THERE IS ADDITIONAL DATA FOR SPECIFIC ATHLETES. BECAUSE IT UNCLEAR AS TO WHICH RACES
# THESE DATA POINTS BELONG TO AND WHETHER THE FULL INFORMATION FOR THESE RACES HAVE BEEN INCLUDED, THIS PART OF THE DATA 
# WILL BE OMMITTED FROM THE PAPER

# We need to drop the last (5125 - 4269) rows
N = 5125 - 4267
df = df.iloc[:-N , :]
df

,Season,Series,City,Country,Year,Month,Day,Distance,Round,Group,...,time_lap1,rank_lap2,time_lap2,rank_lap3,time_lap3,rank_lap4,time_lap4,rank_lap5,time_lap5,Time_Event
0,2012/2013,European Championships,Malmo,SWE,2013,1,18,500m,Preliminaries,1,...,7.52,1.0,9.22,1.0,8.76,1.0,8.87,1.0,9.09,2013.046575
1,2012/2013,European Championships,Malmo,SWE,2013,1,18,500m,Preliminaries,1,...,7.44,2.0,9.31,2.0,8.89,2.0,8.88,2.0,9.27,2013.046575
2,2012/2013,European Championships,Malmo,SWE,2013,1,18,500m,Preliminaries,1,...,7.64,3.0,9.74,3.0,9.04,3.0,9.05,3.0,9.21,2013.046575
3,2012/2013,European Championships,Malmo,SWE,2013,1,18,500m,Preliminaries,1,...,7.46,4.0,9.93,4.0,9.50,4.0,9.67,4.0,9.91,2013.046575
4,2012/2013,European Championships,Malmo,SWE,2013,1,18,500m,Preliminaries,2,...,7.08,2.0,9.18,2.0,8.76,2.0,8.86,1.0,8.90,2013.046575
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4262,2015/2016,World Cup,Dordrecht,NED,2016,2,12,500m,FinalA,1,...,7.15,5.0,8.82,3.0,8.30,4.0,8.45,4.0,8.76,2016.113470
4263,2015/2016,World Cup,Dordrecht,NED,2016,2,12,500m,FinalA,1,...,6.94,3.0,8.83,5.0,18.89,5.0,13.46,5.0,15.48,2016.113470
4264,2015/2016,World Cup,Dordrecht,NED,2016,2,12,500m,FinalA,1,...,7.09,4.0,8.71,6.0,41.81,6.0,28.73,NaN,NaN,2016.113470
4265,2015/2016,World Cup,Dordrecht,NED,2016,2,12,500m,FinalB,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016.113470


In [235]:
# CERTAIN RACES WILL BE REMOVED TO GET A MORE "REPRESENTATIVE" DATASET:

# Races where there were penalties and/or there was(were) skaters that did not start/finish 

for index, row in df.iterrows():
    if (row['Qualification'] == "PEN" or row['Qualification'] == "DNF" or row['Qualification'] == "DNS"):
        df.drop(labels=(index), axis =0, inplace=True)
df = df.reset_index()

# Races where there were < or > 4 skaters: 
# a "race label" is assigned to each skater to determine which race they participated in and races whose labels are 
# counted more or less than 4 times are removed from the dataset

arr = [None]*df['City'].count()
len(arr)
label = 0
for index, row in df.iterrows():
    if (row['Rank_In_Group'] == 1):
        label=label+1
        arr[index] = label
    else:
        arr[index] = label
        
df['Race_label'] = arr

value_counts = df['Race_label'].value_counts()
to_remove = value_counts[value_counts != 4].index
df = df[~df.Race_label.isin(to_remove)]
df = df.reset_index()

# After this process, we are left with 641 races

In [264]:
# ADDING LABEL FOR CATEGORICAL VARIABLE (INTERMEDIATE RANKS COMBINATION)

arrCatLabel = [None]*df['City'].count()
for index, row in df.iterrows():  
    arrCatLabel[index] = int(df.iloc[index]["rank_lap1"])
    

ValueError: cannot convert float NaN to integer

2564